In [1]:
import csv
from cv2 import imread,cvtColor
import cv2
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np

In [2]:
file = open('driver_imgs_list.csv')
lst = csv.reader(file)
file_list = []
for i in lst:
    file_list.append(i)
file_list.pop(0)
w = 240
h = 320
input_size = w * h

In [3]:
def read_file(file):
    img = cv2.imread('train/'+file[1]+'/'+file[2],0)
    img = cv2.resize(img,(w,h), interpolation=cv2.INTER_LINEAR)
    img = img.astype(np.float32)/255.0
    label = int(file[1][1])
    return img.reshape([1,-1]),label
def show(array):
    plt.figure()
    plt.imshow(array.reshape([h,w]), cmap="gray")
    plt.show()
def get_test_batch(batch_size,start):
    batch = np.zeros((batch_size,w*h))
    k = 0
    label = np.zeros((batch_size,10))
    for i in range(start,start+batch_size):
        try:
            batch[k:k+1,:],lab = read_file(file_list[i])
        except:
            print('aaaaa',i)
        label[k,lab] = 1
        k+=1
    return batch,label
def get_batch(batch_size):
    j = np.random.randint(0,20788,batch_size)
    batch = np.zeros((batch_size,w*h))
    k = 0
    label = np.zeros((batch_size,10))
    for i in j:
        batch[k:k+1,:],lab = read_file(file_list[i])
        label[k,lab] = 1
        k+=1
    return batch,label

In [4]:
num_steps = 50000
display_step = 200
batch_size = 128

In [5]:
h_0_size = 10
h_1_size =  int ( w * h / 100) 
h_2_size = int ( w * h / 80) 
h_3_size = int ( w * h / 50) 
input_size = w * h

In [6]:
x = tf.placeholder (dtype=tf.float32,shape=[None, input_size])
y = tf.placeholder (dtype=tf.float32,shape=[None, 10])
lr = tf.placeholder(dtype=tf.float32,shape=[])
dr = tf.placeholder(dtype=tf.float32,shape=[])

In [7]:
h_3 = tf.layers.dense(inputs=x,units=h_3_size,activation=tf.nn.tanh)
d_h_3 = tf.nn.dropout(h_3,dr)
h_2 = tf.layers.dense(inputs=d_h_3,units=h_2_size,activation=tf.nn.tanh)
d_h_2 = tf.nn.dropout(h_2,dr)
h_1 = tf.layers.dense(inputs=d_h_2,units=h_1_size,activation=tf.nn.relu)
d_h_1 = tf.nn.dropout(h_1,dr)
h_0 = tf.layers.dense(inputs=d_h_1,units=10)
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=y, logits=h_0))
optimizer = tf.train.AdamOptimizer(learning_rate=lr).minimize(loss)
correctPred = tf.equal(tf.argmax(h_0,1), tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correctPred, tf.float32))

In [8]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [ ]:
for i in range(1,30*100):
    x_,y_ = get_batch(batch_size)
    sess.run(optimizer,feed_dict={x:x_,y:y_,lr:0.001,dr:0.5})
    if i % 100 == 0:
        x_,y_ = get_batch(batch_size)
        l,acc = sess.run([loss,accuracy],feed_dict={x:x_,y:y_,dr:1})
        print(l,acc)

2.299449 0.1328125
2.3009262 0.09375
2.2947316 0.1484375


In [ ]:
k = 0
total_acc = 0
for i in range(20760,22423,batch_size):
    k += 1
    x_,y_ = get_test_batch(batch_size,i)    
    l,acc = sess.run([loss,accuracy],feed_dict={x:x_,y:y_,dr:1})
    total_acc += acc
    print(l,acc)
total_acc / k